# 11. Lab 11-3 Class , Layers, Ensemble
### 2018.09.30(일)

## Intro

Lab 11-1, Lab11-2에서 CNN을 TensorFlow 로 구현하는 것을 배움

그런데, 이를 python 으로 Class를 만들어 활용할 수 있다.

## Python Class 로 Model 구현하기


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# Lab11-2 에서 구현한  Neural Network 를 Model 이라는 Class  로 구현
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    # Instance 생성 즉시 _build_net() 실행
    def _build_net(self):
        with tf.variable_scope(self.name):
            # input placeholders
            self.X = tf.placeholder(tf.float32, [None, 784])
            # img 28x28x1 (black/white)
            x_img = tf.reshape(self.X, [-1,28,28,1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # L1 ImgIn shape = (?,28,28,1)
            W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
            L1 = tf.nn.conv2d(x_img, W1, strides=[1, 1, 1, 1], padding='SAME')
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
            
            # L2 ImgIn shape=(?, 14, 14, 32)
            W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
            L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            L2 = tf.reshape(L2, [-1,7*7*64])
        
            # Final FC 7x7x64 inputs -> 10outputs
            W3 = tf.get_variable("W3",shape = [7*7*64,10],initializer = tf.contrib.layers.xavier_initializer_conv2d())
            b = tf.Variable(tf.random_normal([10]))
            self.hypothesis = tf.matmul(L2, W3) + b
            # hypothesis 도출된 결과값 ->  hypothesis 가 됨
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.hypothesis, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
    def predict(self, x_test):
        return self.sess.run(self.logits, feed_dict = {self.X: x_test})
    
    def get_accuracy(self, x_test, y_test):
        return self.sess.run(self.accuracy, feed_dict = {self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})
    def train(self, x_data, y_data):
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X: x_data, self.Y: y_data})

    

In [6]:
# initialize
sess = tf.Session()
m1 = Model(sess, "m1")
sess.run(tf.global_variables_initializer())
print("Learning Started!")

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch
        if i % 200 == 0:
            print(i, "cost = ", "{:.9f}".format(avg_cost))
    print("Epoch : ", "%04d" % (epoch + 1), "cost = ", "{:.9f}".format(avg_cost))
print("Learning Finished!")

Learning Started!
0 cost =  0.005760964
200 cost =  0.281179057
400 cost =  0.355469978
Epoch :  0001 cost =  0.390876303
0 cost =  0.000209022
200 cost =  0.042439802
400 cost =  0.080304348
Epoch :  0002 cost =  0.104989893
0 cost =  0.000146575
200 cost =  0.028917404
400 cost =  0.055873069
Epoch :  0003 cost =  0.076718765
0 cost =  0.000102903
200 cost =  0.022107135
400 cost =  0.046871091
Epoch :  0004 cost =  0.063688640
0 cost =  0.000051409
200 cost =  0.020386087
400 cost =  0.040524266
Epoch :  0005 cost =  0.054968983
0 cost =  0.000079861
200 cost =  0.017327518
400 cost =  0.034319726
Epoch :  0006 cost =  0.047693357
0 cost =  0.000099293
200 cost =  0.014709356
400 cost =  0.030269045
Epoch :  0007 cost =  0.042150593
0 cost =  0.000018267
200 cost =  0.012889810
400 cost =  0.025626990
Epoch :  0008 cost =  0.036611897
0 cost =  0.000033029
200 cost =  0.011522821
400 cost =  0.024854334
Epoch :  0009 cost =  0.033021194
0 cost =  0.000043532
200 cost =  0.009875815


## Tensorflow 의 layers   라는 Library  를 이용
![lec11_30](../../img/lec11_30.png)

이를 이용한다면, 숫자가 많이 나와서 복잡하게 계산해야 했던 것을 아주 단순하고 빠르게 처리할 수 있다.

--> High level  (많은 것들을 이미 구현한) 의 library

(ex. layers.dense 는 fully connected layer를 한번에 구현해 준다.)

In [ ]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])

            # img 28x28x1 (black/white), Input Layer
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            # filter 의 개수와 kernel size를 각각 주고, reLU 임을 같이 준다.
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,rate=0.3, training=self.training)
            # dropout의 경우, training이 만약 False로 잡힌다면 keep_prob는 자동으로 1이되어 문제를 예방해놧다.
            
            #  Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,rate=0.3, training=self.training)

            # Layer #3
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],padding="same", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,rate=0.3, training=self.training)

            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat,units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)
            # dense 라는 함수 한번에 fullyconected layer를 한번에 끝냄
            
            # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dropout4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: x_test,self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.training: training})


# Ensemble
여러개의 모델을 조합해서 더 나은 모델을 만들어 사용

여러개의 독립된 모델을  training하여, New data가 나오면 각각의 모델에서 prediction 하여, 그 예측 결과를 meta-classifier하여 결과를 도출

-> 상당히 우수한 결과가 나옴

![lec11_31](../../img/lec11_31.png)

### Ensemble training
#### 앞서 만든  Model을 사용해야 한다.

In [7]:
# initialize
sess = tf.Session()

models = []
#각각 만든 model을 저장하는 list를 만든다.
num_models = 7
for m in range(num_models):
    models.append(Model(sess, "model" + str(m))) #model을 만들어 list에 각각 저장
sess.run(tf.global_variables_initializer())


In [8]:
print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        # 동일한 batch를 가지고학습시킨다.
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        # 기존에는 모델을 하나씩 학습시켰었는데, 이제는 모델을 각각 학습시킨다.(동일한 데이터로)
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')


Learning Started!
Epoch: 0001 cost = [0.34042073 0.37761572]
Epoch: 0002 cost = [0.09855702 0.10618883]
Epoch: 0003 cost = [0.07170656 0.07652393]
Epoch: 0004 cost = [0.05833044 0.06238911]
Epoch: 0005 cost = [0.0498276  0.05185397]
Epoch: 0006 cost = [0.04443941 0.04674543]
Epoch: 0007 cost = [0.03894265 0.03983511]
Epoch: 0008 cost = [0.03400787 0.03524532]
Epoch: 0009 cost = [0.03064473 0.03137245]
Epoch: 0010 cost = [0.02726805 0.02802383]
Epoch: 0011 cost = [0.02517654 0.02531697]
Epoch: 0012 cost = [0.02163124 0.0220168 ]
Epoch: 0013 cost = [0.02018246 0.02086042]
Epoch: 0014 cost = [0.01681946 0.01780215]
Epoch: 0015 cost = [0.01333212 0.01410609]
Learning Finished!


### 학습의 결과로 각각 다른 7개의 model 이 생성될 것.

__이렇게 도출된 model 을 합치는 방법은 여러가지가 있지만, 여기서는 다 합치는 방법으로 구현할 것.__

1. 구현된 모델들은 각각 softmax로 구현되어있다.
2. 그 softmax는 각각의 label별로 확률을 뽑아내는 것 ( ex. 0이 될 확률 ~ 1이되 확률 ~ ...)
3. 모델 각각은 그 확률들을 다르게 계산했을 것.
4. 그러면, 그 확률들을 합쳐준다. (ex. 모델별로 각각 0이 될 확률의 합, ... )
5. 우리는  그 중에서 argmax로 가장 큰 값을 선택하겠다는 간단한 아이디어
![lec11_32](../../img/lec11_32.png)

In [ ]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10]) # -> 예측의 결과값을 저장할 list를 만들어 준다. : testset의 data각각, hot-one encoding 의 결과를 저장

for m_idx, m in enumerate(models): # loop를 돌며 model을 하나씩 꺼내어 test의 accuracy를 print
    print(m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p # 예측한 확률들을 다 더해준다. : prediction에는 최종 합이 된 값을 저장

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))ㅡ


### 결과를 보면, 각각의 model에서 예측한 Accuracy보다 더 높은 정확도를 갖는 Ensemble 모델이 도출되는 것을 알 수 있다.